# <font color='red'>Lista de Espera por Leitos de UTI no Distrito Federal - 12/03/2020, 17:05:05</font>
### Fonte dos Dados Abertos: https://salasit.saude.df.gov.br/lista-de-uti/#

In [1]:
# Importando as bibliotecas do Python

import pandas as pd
from datetime import datetime
import locale
import glob
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Lendo o dataset e convertendo em um dataframe com Pandas
df = pd.read_csv('C:\PROJETOS_GIT_DEV\Lista_Espera_Leitos_UTI\Arquivos\LISTA_UTI_12_03_2020.csv', sep='|')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
Prioridade          80 non-null object
Data                80 non-null object
Hora                80 non-null object
N* da Passagem      80 non-null object
Subtipo de Leito    80 non-null object
dtypes: object(5)
memory usage: 3.2+ KB


In [4]:
# Verificando a quantidade de valores nulos nas colunas
df.isnull().sum()

Prioridade          0
Data                0
Hora                0
N* da Passagem      0
Subtipo de Leito    0
dtype: int64

In [5]:
# Quantidade de registros
print('Quantidade de registros:', len(df.index))

Quantidade de registros: 80


In [6]:
df.head()

,Prioridade,Data,Hora,N* da Passagem,Subtipo de Leito
0,1,12/03/2020,00:14,I2586063,Pediátrico
1,1,12/03/2020,00:54,I2586135,Neonatal
2,1,12/03/2020,01:09,I2586138,Adulto
3,1,12/03/2020,05:13,I2583210,Coronariano
4,1,12/03/2020,07:57,I2586257,Não Informado


In [7]:
# Renomeando colunas
df = df.rename(columns={'N* da Passagem': 'NU_Passagem'})
df = df.rename(columns={'Subtipo de Leito': 'Subtipo_Leito'})

In [8]:
# Concatenando as colunas 'Data' e 'Hora'
df['Data_Hora'] = df['Data'] + ' ' + df['Hora']

In [9]:
del df['Data']
del df['Hora']

In [10]:
df.head()

,Prioridade,NU_Passagem,Subtipo_Leito,Data_Hora
0,1,I2586063,Pediátrico,12/03/2020 00:14
1,1,I2586135,Neonatal,12/03/2020 00:54
2,1,I2586138,Adulto,12/03/2020 01:09
3,1,I2583210,Coronariano,12/03/2020 05:13
4,1,I2586257,Não Informado,12/03/2020 07:57


In [11]:
# Convertendo a coluna 'Data_Hora' em datetime
df['Data_Hora'] =  pd.to_datetime(df['Data_Hora'], format='%d/%m/%Y %H:%M')

In [12]:
df.head()

,Prioridade,NU_Passagem,Subtipo_Leito,Data_Hora
0,1,I2586063,Pediátrico,2020-03-12 00:14:00
1,1,I2586135,Neonatal,2020-03-12 00:54:00
2,1,I2586138,Adulto,2020-03-12 01:09:00
3,1,I2583210,Coronariano,2020-03-12 05:13:00
4,1,I2586257,Não Informado,2020-03-12 07:57:00


In [13]:
df.dtypes

Prioridade               object
NU_Passagem              object
Subtipo_Leito            object
Data_Hora        datetime64[ns]
dtype: object

In [14]:
# Criando a coluna de Hora com tipo time e valor cheio
df['Hora'] = df['Data_Hora'].apply(lambda time: time.hour)

In [15]:
# Definindo o id do mês
df['Id_Mes'] = df['Data_Hora'].dt.month

In [16]:
# Definindo o nome do mês
df['Nome_Mes'] = df.Data_Hora.dt.month_name(locale = 'Portuguese') 

In [17]:
# Definindo o id do dia da semana
df['Id_Dia_Semana'] = df['Data_Hora'].dt.weekday    

In [18]:
# Definindo o nome do dia da semana
df['Nome_Dia_Semana'] = df.Data_Hora.dt.day_name(locale='pt')

In [19]:
# Verificando o id_dia_semana
qw=pd.unique(df[['Id_Dia_Semana']].values.ravel()) 
print(qw)

[3 1 2 5 0 4 6]


In [20]:
# Função para ajustar o id do dia da semana
def ajusta_id_dia_semana(id_dia):
    if id_dia == 6: #domingo
        novo_id = 1
    elif id_dia == 0: #segunda
        novo_id = 2
    elif id_dia == 1: #terça
        novo_id = 3
    elif id_dia == 2: #quarta
        novo_id = 4
    elif id_dia == 3: #quinta
        novo_id = 5
    elif id_dia == 4: #sexta
        novo_id = 6
    elif id_dia == 5: #sábado
        novo_id = 7
    return novo_id

In [21]:
# Ajustando os valores da coluna Id_Dia_Semana
df['Id_Dia_Semana'] = df.Id_Dia_Semana.apply(ajusta_id_dia_semana)

In [22]:
df.head()

,Prioridade,NU_Passagem,Subtipo_Leito,Data_Hora,Hora,Id_Mes,Nome_Mes,Id_Dia_Semana,Nome_Dia_Semana
0,1,I2586063,Pediátrico,2020-03-12 00:14:00,0,3,Março,5,Quinta-feira
1,1,I2586135,Neonatal,2020-03-12 00:54:00,0,3,Março,5,Quinta-feira
2,1,I2586138,Adulto,2020-03-12 01:09:00,1,3,Março,5,Quinta-feira
3,1,I2583210,Coronariano,2020-03-12 05:13:00,5,3,Março,5,Quinta-feira
4,1,I2586257,Não Informado,2020-03-12 07:57:00,7,3,Março,5,Quinta-feira


In [23]:
# Função para definir o período do dia
def define_nome_periodo(hora):
    madrugada = [0, 1, 2, 3, 4, 5]
    manha = [6, 7, 8, 9, 10, 11, 12]
    tarde = [13, 14, 15, 16, 17, 18]
    noite = [19, 20, 21, 22, 23]
    if hora in madrugada:
        nome_periodo = 'Madrugada'
    elif hora in manha:
        nome_periodo = 'Manhã'
    elif hora in tarde:
        nome_periodo = 'Tarde'
    elif hora in noite:
        nome_periodo = 'Noite'
    return nome_periodo

In [24]:
# Função para definir o id do período do dia
def define_id_periodo(nome_periodo):
    if nome_periodo == 'Madrugada':
        id_periodo = 1
    elif nome_periodo == 'Manhã':
        id_periodo = 2
    elif nome_periodo == 'Tarde':
        id_periodo = 3
    elif nome_periodo == 'Noite':
        id_periodo = 4
    return id_periodo

In [25]:
df['Nome_Periodo'] = df.Hora.apply(define_nome_periodo)

In [26]:
df['Id_Periodo'] = df.Nome_Periodo.apply(define_id_periodo)

In [27]:
df.head()

,Prioridade,NU_Passagem,Subtipo_Leito,Data_Hora,Hora,Id_Mes,Nome_Mes,Id_Dia_Semana,Nome_Dia_Semana,Nome_Periodo,Id_Periodo
0,1,I2586063,Pediátrico,2020-03-12 00:14:00,0,3,Março,5,Quinta-feira,Madrugada,1
1,1,I2586135,Neonatal,2020-03-12 00:54:00,0,3,Março,5,Quinta-feira,Madrugada,1
2,1,I2586138,Adulto,2020-03-12 01:09:00,1,3,Março,5,Quinta-feira,Madrugada,1
3,1,I2583210,Coronariano,2020-03-12 05:13:00,5,3,Março,5,Quinta-feira,Madrugada,1
4,1,I2586257,Não Informado,2020-03-12 07:57:00,7,3,Março,5,Quinta-feira,Manhã,2


In [28]:
# Definindo o id do mês
df['Dia'] = df['Data_Hora'].dt.day

In [29]:
df.head()

,Prioridade,NU_Passagem,Subtipo_Leito,Data_Hora,Hora,Id_Mes,Nome_Mes,Id_Dia_Semana,Nome_Dia_Semana,Nome_Periodo,Id_Periodo,Dia
0,1,I2586063,Pediátrico,2020-03-12 00:14:00,0,3,Março,5,Quinta-feira,Madrugada,1,12
1,1,I2586135,Neonatal,2020-03-12 00:54:00,0,3,Março,5,Quinta-feira,Madrugada,1,12
2,1,I2586138,Adulto,2020-03-12 01:09:00,1,3,Março,5,Quinta-feira,Madrugada,1,12
3,1,I2583210,Coronariano,2020-03-12 05:13:00,5,3,Março,5,Quinta-feira,Madrugada,1,12
4,1,I2586257,Não Informado,2020-03-12 07:57:00,7,3,Março,5,Quinta-feira,Manhã,2,12


### Estatísticas


In [30]:
df.describe()

,Hora,Id_Mes,Id_Dia_Semana,Id_Periodo,Dia
count,80.000000,80.000000,80.000000,80.000000,80.000000
mean,12.262500,2.975000,3.925000,2.512500,11.762500
std,6.227224,1.078853,1.589324,0.941232,6.455777
min,0.000000,2.000000,1.000000,1.000000,3.000000
25%,9.000000,3.000000,3.000000,2.000000,9.000000
50%,12.000000,3.000000,4.000000,2.000000,10.000000
75%,17.000000,3.000000,5.000000,3.000000,12.000000
max,23.000000,12.000000,7.000000,4.000000,29.000000


In [31]:
df['Prioridade'].value_counts()

2     26
1     24
3     22
4A     5
4B     3
Name: Prioridade, dtype: int64

In [32]:
df['Subtipo_Leito'].value_counts()

Adulto           37
Não Informado    32
Coronariano       8
Pediátrico        2
Neonatal          1
Name: Subtipo_Leito, dtype: int64

In [33]:
# Gerando CSV para mostrar os dados no PowerBI
df.to_csv('lista_espera_leitos_uti.csv')